1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [11]:
import pandas as pd
import numpy as np
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/
file = "./data_000637.txt"
N=59000
data = pd.read_csv(file, sep=',',nrows=N)
print(data)
data.describe()

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0         1     0          123  3869200167        2374        26
1         1     0          124  3869200167        2374        27
2         1     0           63  3869200167        2553        28
3         1     0           64  3869200167        2558        19
4         1     0           64  3869200167        2760        25
...     ...   ...          ...         ...         ...       ...
58995     1     1           61  3869200754         110         0
58996     1     1          139  3869200754         114         0
58997     1     1           63  3869200754         112        28
58998     1     0          123  3869200754         165        14
58999     1     0           92  3869200754         167         9

[59000 rows x 6 columns]


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
count,59000.0,59000.000000,59000.000000,5.900000e+04,59000.000000,59000.000000
mean,1.0,0.293373,56.101220,3.869200e+09,1788.442119,13.351305
std,0.0,0.455312,39.098571,1.692416e+02,1025.448479,9.188582
min,1.0,0.000000,1.000000,3.869200e+09,0.000000,0.000000
25%,1.0,0.000000,26.000000,3.869200e+09,893.000000,5.000000
50%,1.0,0.000000,57.000000,3.869200e+09,1792.000000,13.000000
75%,1.0,1.000000,64.000000,3.869201e+09,2676.000000,21.000000
max,1.0,1.000000,139.000000,3.869201e+09,3563.000000,29.000000


In [10]:
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
58995,1,1,61,3869200754,110,0
58996,1,1,139,3869200754,114,0
58997,1,1,63,3869200754,112,28
58998,1,0,123,3869200754,165,14


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [17]:
for i in range (N):
    if data.iloc[i]['BX_COUNTER'] == 0:
        print(" BX counter resets at row: ", i, " The value: ", data.iloc[i]['BX_COUNTER'])
        print("Maximum value before being reset to 0:", data.iloc[i-1]['BX_COUNTER'], " Row: ", i-1)
        x = data.iloc[i-1]['BX_COUNTER']
        break
print("x =", x)

 BX counter resets at row:  2894  The value:  0
Maximum value before being reset to 0: 3563  Row:  2893
x = 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [21]:
import datetime as td
data = data.assign(ABSOLUTE_TIME=data['BX_COUNTER'] + data['TDC_MEAS'] +data['ORBIT_CNT'])
data/10**9

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABSOLUTE_TIME
0,1.000000e-09,0.000000e+00,1.230000e-07,3.869200,2.374000e-06,2.600000e-08,3.869203
1,1.000000e-09,0.000000e+00,1.240000e-07,3.869200,2.374000e-06,2.700000e-08,3.869203
2,1.000000e-09,0.000000e+00,6.300000e-08,3.869200,2.553000e-06,2.800000e-08,3.869203
3,1.000000e-09,0.000000e+00,6.400000e-08,3.869200,2.558000e-06,1.900000e-08,3.869203
4,1.000000e-09,0.000000e+00,6.400000e-08,3.869200,2.760000e-06,2.500000e-08,3.869203
...,...,...,...,...,...,...,...
58995,1.000000e-09,1.000000e-09,6.100000e-08,3.869201,1.100000e-07,0.000000e+00,3.869201
58996,1.000000e-09,1.000000e-09,1.390000e-07,3.869201,1.140000e-07,0.000000e+00,3.869201
58997,1.000000e-09,1.000000e-09,6.300000e-08,3.869201,1.120000e-07,2.800000e-08,3.869201
58998,1.000000e-09,0.000000e+00,1.230000e-07,3.869201,1.650000e-07,1.400000e-08,3.869201


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [32]:
data['DURATION'] = pd.to_timedelta(data['ABSOLUTE_TIME'])
print("The duration for each:", data['DURATION'][0:5].diff())
print("The duration for the whole data:", (data['DURATION'].max() - data['DURATION'].min()))

The duration for each: 0                           NaT
1     0 days 00:00:00.000000001
2     0 days 00:00:00.000000180
3   -1 days +23:59:59.999999996
4     0 days 00:00:00.000000208
Name: DURATION, dtype: timedelta64[ns]
The duration for the whole data: 0 days 00:00:00.000004139


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [54]:
print(data.groupby('HEAD')['TDC_CHANNEL'].nlargest(3)) #The top 3 TDC_CHANNELS

HEAD    
1     7     139
      15    139
      30    139
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [40]:
count = data.ORBIT_CNT.unique().size
print(str(count)) #number of non-empty orbits

588


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [46]:
uo = pd.DataFrame(data[data['TDC_CHANNEL'] == 139]).drop_duplicates(subset=['ORBIT_CNT'], inplace=False)
print("\nThe number of unique orbits with at least one measurement from TDC_CHANNEL = 139 is ", len(uo))


The number of unique orbits with at least one measurement from TDC_CHANNEL = 139 is  587


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [49]:
FPGA1 = pd.Series(data[data['FPGA'] == 0]['TDC_CHANNEL'].value_counts())
print(FPGA1) #Series 1
FPGA2 = pd.Series(data[data['FPGA'] == 1]['TDC_CHANNEL'].value_counts())
print(FPGA2) #Series 2

139    3446
64     2919
63     2892
61     2198
62     2188
       ... 
104      22
103      20
106      19
101      16
67        8
Name: TDC_CHANNEL, Length: 118, dtype: int64
139    1462
2      1429
1      1261
4      1171
3       999
       ... 
94        4
78        4
86        4
91        2
130       2
Name: TDC_CHANNEL, Length: 129, dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.